# Introducing Rendervous  

Rendervous is a project built on Vulkan and PyTorch that models processes—potentially graphics-related—as differentiable maps whenever possible. These maps define parametric, vector-valued functions as follows:  

$$  
f_\theta: \mathcal{R}^n \rightarrow \mathcal{R}^m \quad \text{where} \quad y = f_\theta(x)  
$$  

Each map includes a forward method to compute $y$ and a backward mechanism to propagate gradients, $\partial L / \partial y$, back to both $x$ and $\theta$.

Unlike conventional tensor-based differentiation frameworks, Rendervous focuses on operator-level abstraction rather than an operation-result graph. Depending on the map's specifics, gradient computation may either be automated or involve replaying the forward evaluation. This design requires careful consideration when combining maps to balance execution complexity effectively.

Let's start importing necessary libraries.

In [1]:
try: # install all dependencies in colab 
    import google.colab
    !sudo apt-get update -y
    !sudo apt-get install -y libnvidia-gl-555 vulkan-tools glslang-tools vulkan-validationlayers-dev
    !pip install pyav==13.1.0
    !pip install git+https://github.com/rendervous/rendervous_project.git
except:
    print("Executing locally")

Executing locally


In [1]:
import rendervous as rdv
import torch

We will start with the simplest map in Rendervous: a constant map. This map represents functions in the form:  

$$  
f_\theta(x) := \theta  
$$  

For example: 

In [2]:
my_consmap = rdv.const[1.0, 2.0]

Some maps can be defined in a generic form. For instance, the map `my_consmap` has a fixed output dimension of 2, but its input dimension can vary. The method `cast` allows you to set specific input and output dimensions, returning a new map that is linked to the same parameters or sub-maps.

In [3]:
my_non_generic_consmap = my_consmap.cast(input_dim=4)
print(my_non_generic_consmap(torch.rand(10, 2, device=rdv.device())))

AssertionError: Map input dimension and tensor last dim missmatch

In [4]:
print(my_non_generic_consmap(torch.rand(10, 4, device=rdv.device())))

tensor([[1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.]], device='cuda:0', grad_fn=<AutogradMapFunctionBackward>)


If the map is generic at the input it can be evaluated and will take the input dimension from the last dimension of the argument.

In [5]:
print(my_consmap(torch.rand(10, 2, device=rdv.device())))  # no problem here.

tensor([[1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.]], device='cuda:0', grad_fn=<AutogradMapFunctionBackward>)


Some special constants (non-trainable) can be instantiated directly. 

In [7]:
zero = rdv.ZERO
one = rdv.ONE

Another interested behaviour in rendervous is the promotion of scalar-valued functions (i.e., functions with output dimension 1).
Those functions behave as if they were generic in the output since they can be cast to any output dimension.

In [8]:
print(zero.cast(output_dim=3)(torch.rand(5, 4)))

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], device='cuda:0', grad_fn=<AutogradMapFunctionBackward>)


In case a map with generic output is evaluated, a single value output is assumed. 

In [9]:
print(one(torch.rand(5,4)))

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:0', grad_fn=<AutogradMapFunctionBackward>)


## Operations  

Unlike tensor-based operations and automatic differentiation, in Rendervous, operators applied to maps create new maps. For example, the following creates a map representing the addition of two maps. Specifically, it defines a map that extracts the direction $\omega$ and the position $x$ from a ray input in the form $x|\omega$. The resulting map computes the ray's end-point at a distance of $1$ (assuming the direction is normalized).  


In [10]:
my_map = rdv.ray_direction + rdv.ray_position

In [11]:
rays = torch.randn(10, 6)
print(rays)
ray_ends = my_map(rays)
print(ray_ends)

tensor([[-0.4744, -0.1577, -1.0514, -0.5573, -1.2427, -0.2870],
        [ 0.4056, -0.0103,  0.5929,  0.6980, -1.5349, -0.1465],
        [ 0.3396,  0.1659,  0.6505,  0.1052, -0.0693, -0.6165],
        [-0.2353, -1.8952,  0.8709,  0.0129,  0.7126,  0.7095],
        [ 0.2671,  0.0910,  0.9266,  0.6425, -1.1164, -0.4757],
        [-0.1114, -1.0496,  1.1904,  0.6152,  0.2952, -0.7792],
        [-1.1919,  0.3965,  0.4316,  0.2657,  0.1771, -0.1510],
        [-0.6620, -1.3776, -1.1333,  0.4385,  0.4997,  0.8710],
        [ 0.1856,  0.3566,  0.1963,  1.2096,  1.4897,  0.5286],
        [ 0.1520,  2.1677,  1.0299,  1.5249,  0.0660,  0.2026]])
tensor([[-1.0316, -1.4004, -1.3384],
        [ 1.1036, -1.5452,  0.4464],
        [ 0.4448,  0.0966,  0.0340],
        [-0.2224, -1.1827,  1.5805],
        [ 0.9095, -1.0255,  0.4509],
        [ 0.5038, -0.7544,  0.4112],
        [-0.9262,  0.5736,  0.2806],
        [-0.2234, -0.8779, -0.2623],
        [ 1.3952,  1.8463,  0.7248],
        [ 1.6769,  2.2337,

In [12]:
rays[:,0:3] + rays[:,3:6]

tensor([[-1.0316, -1.4004, -1.3384],
        [ 1.1036, -1.5452,  0.4464],
        [ 0.4448,  0.0966,  0.0340],
        [-0.2224, -1.1827,  1.5805],
        [ 0.9095, -1.0255,  0.4509],
        [ 0.5038, -0.7544,  0.4112],
        [-0.9262,  0.5736,  0.2806],
        [-0.2234, -0.8779, -0.2623],
        [ 1.3952,  1.8463,  0.7248],
        [ 1.6769,  2.2337,  1.2325]])

Also, when operating a map with a scalar value it is converted to a constant map automatically. Notice the use of `|` as concat operator.

In [13]:
opposed_ray = rdv.ray_position | (-1 * rdv.ray_direction)
print(opposed_ray(rays))

tensor([[-0.4744, -0.1577, -1.0514,  0.5573,  1.2427,  0.2870],
        [ 0.4056, -0.0103,  0.5929, -0.6980,  1.5349,  0.1465],
        [ 0.3396,  0.1659,  0.6505, -0.1052,  0.0693,  0.6165],
        [-0.2353, -1.8952,  0.8709, -0.0129, -0.7126, -0.7095],
        [ 0.2671,  0.0910,  0.9266, -0.6425,  1.1164,  0.4757],
        [-0.1114, -1.0496,  1.1904, -0.6152, -0.2952,  0.7792],
        [-1.1919,  0.3965,  0.4316, -0.2657, -0.1771,  0.1510],
        [-0.6620, -1.3776, -1.1333, -0.4385, -0.4997, -0.8710],
        [ 0.1856,  0.3566,  0.1963, -1.2096, -1.4897, -0.5286],
        [ 0.1520,  2.1677,  1.0299, -1.5249, -0.0660, -0.2026]],
       device='cuda:0', grad_fn=<AutogradMapFunctionBackward>)
